## Простейшая предобработка текста (удаление цифр, пунктуации)

In [117]:
import re

# добавим пробелы перед и после знаков препинания, 
# чтобы при дальнейшем удалении пунктуации отдельные слова не сливались в одно
def add_space(text):
    new_text = ''
    for i in text:
        if i in '!"#$&\'()%*+,.:;<=>?@[\\]^_`{|}~':
            new_text += ' ' + i + ' '
        else:
            new_text += i
    return new_text

# удалим все числа (хотя возможно они были бы полезны)
def remove_numbers(text):
    result = re.sub(r'\d+', '', text)
    return result

# удалим знаки препинания
def remove_punctuation(text):
    translator = str.maketrans('', '', '!"#$&%*\'()+,.:;<=>?@[\\]^_`{|}~')
    return text.translate(translator)

# избавимся от лишних пробелов
def remove_whitespace(text):
    return  " ".join(text.split())

def preprocessing_text(text_feature):
    text_feature = text_feature.apply(add_space)
    text_feature = text_feature.apply(lambda x: x.lower()) 
    text_feature = text_feature.apply(remove_numbers)
    text_feature = text_feature.apply(remove_punctuation)
    text_feature = text_feature.apply(remove_whitespace)
    return text_feature

## Преобразование слов в начальную форму
с помощью pymorfy2

In [ ]:
!pip install pymorphy2

     |████████████████████████████████| 61kB 4.9MB/s 
     |████████████████████████████████| 8.2MB 14.5MB/s 


In [ ]:
import pymorphy2
import os
#преобразование в нач.форму слов с использованием pyMorfy2
def f_tokenizer(s):
    morph = pymorphy2.MorphAnalyzer()
    j1 = ' '.join(map(str, s))
    j2 = j1.split()       
    f = []
    for word in j2:
        m = morph.parse(word)
 #       m = [morph.parse(word) for word in j]
        if len(m) != 2:
          wrd = m[0]
          if wrd.tag.POS not in ('NUMR','PREP','CONJ','PRCL','INTJ'):
              f.append(wrd.normal_form)
    return f

In [ ]:
marked_data1 = f_tokenizer(marked_data['item_name'])

In [3]:
## Стеммер Портера - алгоритм не использует баз основ слов, а лишь, применяя последовательно ряд правил, отсекает окончания и суффиксы, основываясь на особенностях языка, в связи с чем работает быстро, но не всегда безошибочно.

In [ ]:
class Porter:
	PERFECTIVEGROUND =  re.compile(u"((ив|ивши|ившись|ыв|ывши|ывшись)|((?<=[ая])(в|вши|вшись)))$")
	REFLEXIVE = re.compile(u"(с[яь])$")
	ADJECTIVE = re.compile(u"(ее|ие|ые|ое|ими|ыми|ей|ий|ый|ой|ем|им|ым|ом|его|ого|ему|ому|их|ых|ую|юю|ая|яя|ою|ею)$")
	PARTICIPLE = re.compile(u"((ивш|ывш|ующ)|((?<=[ая])(ем|нн|вш|ющ|щ)))$")
	VERB = re.compile(u"((ила|ыла|ена|ейте|уйте|ите|или|ыли|ей|уй|ил|ыл|им|ым|ен|ило|ыло|ено|ят|ует|уют|ит|ыт|ены|ить|ыть|ишь|ую|ю)|((?<=[ая])(ла|на|ете|йте|ли|й|л|ем|н|ло|но|ет|ют|ны|ть|ешь|нно)))$")
	NOUN = re.compile(u"(а|ев|ов|ие|ье|е|иями|ями|ами|еи|ии|и|ией|ей|ой|ий|й|иям|ям|ием|ем|ам|ом|о|у|ах|иях|ях|ы|ь|ию|ью|ю|ия|ья|я)$")
	RVRE = re.compile(u"^(.*?[аеиоуыэюя])(.*)$")
	DERIVATIONAL = re.compile(u".*[^аеиоуыэюя]+[аеиоуыэюя].*ость?$")
	DER = re.compile(u"ость?$")
	SUPERLATIVE = re.compile(u"(ейше|ейш)$")
	I = re.compile(u"и$")
	P = re.compile(u"ь$")
	NN = re.compile(u"нн$")

	def stem(word):
		word = word.lower()
		word = word.replace(u'ё', u'е')
		m = re.match(Porter.RVRE, word)
		if m and m.groups():
			pre = m.group(1)
			rv = m.group(2)
			temp = Porter.PERFECTIVEGROUND.sub('', rv, 1)
			if temp == rv:
				rv = Porter.REFLEXIVE.sub('', rv, 1)
				temp = Porter.ADJECTIVE.sub('', rv, 1)
				if temp != rv:
					rv = temp
					rv = Porter.PARTICIPLE.sub('', rv, 1)
				else:
					temp = Porter.VERB.sub('', rv, 1)
					if temp == rv:
						rv = Porter.NOUN.sub('', rv, 1)
					else:
						rv = temp
			else:
				rv = temp
			
			rv = Porter.I.sub('', rv, 1)

			if re.match(Porter.DERIVATIONAL, rv):
				rv = Porter.DER.sub('', rv, 1)

			temp = Porter.P.sub('', rv, 1)
			if temp == rv:
				rv = Porter.SUPERLATIVE.sub('', rv, 1)
				rv = Porter.NN.sub(u'н', rv, 1)
			else:
				rv = temp
			word = pre+rv
		return word
	stem=staticmethod(stem)

In [ ]:
Porter.stem(u'устойчивость')

'устойчив'

In [ ]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [114]:
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = (
        re.sub('[\W]+', ' ', text.lower()) +
        ' '.join(emoticons).replace('-', '')
    )
    return text


def tokenizer(text):
    return text.split()


def tokenizer_porter(text):
    return [Porter.stem(word) for word in text.split()]


def tokenizer_streaming(text):
    text = preprocessor(text)
    stop = stopwords.words('russian')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

def tokenizer_streaming_porter(text):
    text = preprocessor(text)
    stop = stopwords.words('russian')
    tokenized = [Porter.stem(word) for word in text.split() if word not in stop]
    return tokenized

def preprocessing(text_feature):
    text_feature = text_feature.apply(preprocessor)
    #text_feature = text_feature.apply(tokenizer) 
    #text_feature = text_feature.apply(tokenizer_porter)
    text_feature = text_feature.apply(tokenizer_streaming) 
    return text_feature

